In [161]:
import pandas as pd
import anndata as ad
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, csc_matrix
from scipy.io import mmread, mmwrite
import os
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.utils.multiclass import type_of_target

In [162]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [163]:
###seurat

In [ ]:
train_id = "Dataset35"
test_id = "Dataset36"

In [164]:
##读取

In [165]:
accesson=pd.read_csv("..data/" + test_id + "/matrix/Accesson_peaks.csv",sep="\t")
accesson.columns.values[0]="peak"

In [166]:
peaks=pd.read_csv("..data/" + test_id + "/peaks.bed",sep="\t",header=None)

In [167]:
accesson.peak = peaks.iloc[:,0]

In [168]:
accesson

,peak,group
0,chr1-9937-10498,558
1,chr1-180644-181778,46
2,chr1-191326-191849,586
3,chr1-267837-268193,586
4,chr1-586031-586363,586
...,...,...
68958,chrY-11721792-11722236,55
68959,chrY-11747573-11747839,55
68960,chrY-56829443-56829881,55
68961,chrY-56830756-56834738,586


In [170]:
true=pd.read_csv("../results/Seurat/Seurat_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/Seurat/Seurat_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)

In [171]:
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [172]:
true=true.T
pred=pred.T
pred=pred.loc[true.index.values.flatten(),]

In [174]:
accesson=accesson[accesson["peak"].isin(true.index.values.flatten())]

In [175]:
accesson=accesson.reset_index()

In [176]:
index_y =accesson.index.values
accessons,index_x =np.unique(accesson.group,return_inverse=True)
Accesson_gene= coo_matrix((np.ones(shape=(len(index_x),)),(index_x,index_y)))
Accesson_gene_matrix=pd.DataFrame(Accesson_gene.toarray(),columns=true.index.values.flatten())

In [177]:
Accesson_gene_matrix

,chr1-100028620-100029223,chr1-100034488-100034828,chr1-100036915-100039547,chr1-100046586-100047303,chr1-100132285-100133776,chr1-100184590-100184896,chr1-100249022-100250476,chr1-100265455-100266982,chr1-100299590-100299838,chr1-100307011-100307158,...,chrY-11294904-11295988,chrY-11305131-11305629,chrY-11312214-11312892,chrY-11314266-11315078,chrY-11327423-11327918,chrY-11329701-11330187,chrY-11332886-11334319,chrY-56829443-56829881,chrY-56830756-56834738,chrY-56835728-56837353
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix.dot(true)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,4.0,10.0,14.0,22.0,8.0,24.0,27.0,48.0,16.0,...,28.0,12.0,22.0,8.0,3.0,13.0,12.0,4.0,34.0,13.0
2,0.0,0.0,3.0,2.0,2.0,2.0,4.0,2.0,6.0,4.0,...,4.0,6.0,0.0,0.0,0.0,4.0,2.0,0.0,2.0,4.0
3,6.0,0.0,2.0,10.0,19.0,2.0,16.0,16.0,32.0,8.0,...,15.0,11.0,3.0,8.0,0.0,30.0,4.0,12.0,22.0,14.0
4,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
597,6.0,4.0,22.0,47.0,65.0,12.0,32.0,62.0,109.0,36.0,...,53.0,41.0,25.0,17.0,8.0,42.0,15.0,4.0,61.0,36.0
598,2.0,0.0,4.0,6.0,2.0,0.0,8.0,8.0,8.0,4.0,...,10.0,5.0,0.0,0.0,4.0,16.0,8.0,0.0,16.0,4.0


In [179]:
pred_matrix=Accesson_gene_matrix.dot(pred)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,2.867025,1.405876,0.000000,0.747367,0.478907,0.364546,0.138602,0.113756,1.420080,0.000000,...,0.686824,0.337987,0.428702,0.649340,1.618536,1.179916,0.000000,0.000000,0.411475,0.000000
1,8.725651,14.689294,9.563503,18.881998,20.053004,22.374068,17.081371,10.902244,31.143098,10.273902,...,13.541121,8.764246,13.925191,9.514585,34.451997,8.081197,15.171802,4.810677,22.187841,7.235623
2,0.371852,0.628724,0.340077,1.079550,0.918670,2.015191,2.058157,1.103520,4.515991,0.011977,...,2.085085,0.858809,2.023588,1.837053,2.909787,1.878579,0.191364,1.327199,1.482039,3.082526
3,3.140762,8.213493,11.101036,11.866492,16.400224,10.359864,15.685268,6.450042,24.035391,5.138765,...,6.477144,6.025311,7.913669,8.516162,22.995089,7.422903,4.913780,4.261830,17.212109,7.739977
4,1.385665,3.962533,1.841577,0.981377,1.269609,1.023363,0.277204,0.396967,0.842164,0.643246,...,0.673646,0.000000,0.371710,0.613306,0.912218,0.399694,0.048196,0.362829,1.225134,0.276159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056878,0.000000,0.000000,...,0.077615,0.000000,0.196781,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087186
596,0.000000,0.000000,0.075687,0.000000,0.441639,0.094289,0.000000,0.322628,0.024674,0.000000,...,0.000000,0.112662,0.000000,0.072636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
597,21.875515,40.184602,26.887333,35.781586,47.517564,57.615038,38.802869,23.558891,59.086675,18.930606,...,25.673796,20.660200,33.897025,22.281323,71.414812,30.633120,29.988604,12.398627,33.082594,20.213850
598,0.636951,3.392214,3.766726,4.949112,4.284966,9.062962,8.422057,3.175607,9.382054,1.992806,...,1.205105,1.253495,5.345167,3.489043,9.101299,7.018350,3.273766,3.387462,6.967993,1.821739


In [180]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [181]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [182]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.1846571373297192

In [183]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.9971632716572066

In [ ]:
pccs_row_df.to_csv("../results/Accesson/Seurat_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/Seurat_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [185]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
Seurat_cmd_accesson=CMD_dist(corr_pred,corr_true)

In [186]:
Seurat_cmd_accesson

0.01844179154465131

In [187]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
Seurat_cmd_cell=CMD_dist(corr_pred,corr_true)

In [188]:
Seurat_cmd_cell

0.00030720179802223857

In [189]:
##LIGER

In [190]:
true=pd.read_csv("../results/LIGER/LIGER_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/LIGER/LIGER_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)
true=true.T
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()
true=true.loc[pred.index]
pred=pred.loc[:,true.columns]

In [192]:
pred=pred.T

In [193]:
true=true.loc[pred.index]
pred=pred.loc[:,true.columns]

In [195]:
accesson=pd.read_csv("..data/" + test_id + "/matrix/Accesson_peaks.csv",sep="\t")
accesson.columns.values[0]="peak"

In [196]:
index_y =accesson.index.values
accessons,index_x =np.unique(accesson.group,return_inverse=True)
Accesson_gene= coo_matrix((np.ones(shape=(len(index_x),)),(index_x,index_y)))
Accesson_gene_matrix=pd.DataFrame(Accesson_gene.toarray(),columns=true.index.values.flatten())

In [197]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix.dot(true)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,17.0,25.0,41.0,18.0,14.0,35.0,35.0,11.0,...,40.0,32.0,14.0,6.0,6.0,16.0,18.0,8.0,32.0,17.0
2,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,4.0,0.0,...,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
3,6.0,6.0,6.0,26.0,17.0,12.0,10.0,25.0,52.0,15.0,...,14.0,11.0,18.0,13.0,6.0,16.0,4.0,4.0,47.0,20.0
4,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,14.0,6.0,18.0,34.0,54.0,17.0,26.0,52.0,69.0,42.0,...,31.0,47.0,27.0,24.0,10.0,34.0,16.0,8.0,36.0,41.0
598,2.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,2.0,8.0,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [198]:
pred_matrix=Accesson_gene_matrix.dot(pred)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.099999,0.174999,0.200000,0.050000,0.050000,0.100000,0.150000,0.100000,0.100000,0.199999,...,0.249996,0.199999,0.100000,0.099999,0.249998,0.025000,0.149999,0.100000,0.150000,0.249999
1,42.475113,41.399977,51.025043,37.399996,37.925067,39.174988,47.975024,37.824946,47.275029,43.924901,...,37.550056,37.974931,33.224966,36.399988,37.374932,40.250013,43.074949,49.875030,47.274990,47.049974
2,1.550002,1.124999,1.650000,1.250000,1.074998,1.549998,1.575000,1.024998,1.525002,1.149998,...,1.725008,1.174997,0.899997,0.800000,1.024998,1.399998,1.275001,1.524999,1.350001,1.425005
3,36.750104,30.749977,39.200028,29.074966,29.725052,33.049982,38.300025,30.024942,40.350006,33.124929,...,28.325036,29.949931,25.874974,28.024973,29.449943,33.800001,36.074964,39.875022,38.425004,38.599996
4,1.650005,1.199999,1.675001,1.300001,1.275003,1.724999,1.850000,1.824993,2.075003,1.699993,...,1.425005,1.599996,1.199997,1.225000,1.574997,1.500000,2.174998,2.000002,2.099999,2.075002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.050000,0.025000,0.200000,0.049999,0.099999,0.199999,0.200000,0.000000,0.100000,0.200000,...,0.199997,0.124999,0.200000,0.099999,0.199999,0.050000,0.050000,0.100000,0.150000,0.050000
596,0.000000,0.150000,0.075000,0.050001,0.000000,0.050000,0.125000,0.050001,0.000000,0.100000,...,0.250000,0.050000,0.174999,0.050000,0.000000,0.000000,0.199999,0.050000,0.199999,0.199999
597,66.800200,57.774965,68.775079,56.149961,52.500132,60.249973,69.075046,58.849889,71.875051,62.824858,...,53.825054,57.949885,47.574971,54.749948,56.924902,60.500036,62.599952,75.250042,67.824996,68.025014
598,3.024996,3.449996,3.800001,3.374993,2.700001,3.249998,3.675000,2.999988,3.450002,3.899993,...,3.200016,3.024990,2.950001,2.899991,3.249995,3.349995,3.349997,3.475000,3.899997,3.700002


In [199]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

/public/home/maintain/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/public/home/maintain/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [200]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [201]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

nan

In [202]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.9977971896148576

In [203]:
pccs_row_df.to_csv("../results/Accesson/LIGER_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/LIGER_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [204]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
LIGER_cmd_accesson=CMD_dist(corr_pred,corr_true)

In [205]:
LIGER_cmd_accesson

nan

In [206]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
LIGER_cmd_cell=CMD_dist(corr_pred,corr_true)

In [207]:
LIGER_cmd_cell

0.00028709213904398734

In [208]:
# CMAE
true=pd.read_csv("../results/CMAE/CMAE_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/CMAE/CMAE_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)
true=true.T
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [209]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix.dot(true.T)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,17.0,25.0,41.0,18.0,14.0,35.0,35.0,11.0,...,40.0,32.0,14.0,6.0,6.0,16.0,18.0,8.0,32.0,17.0
2,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,4.0,0.0,...,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
3,6.0,6.0,6.0,26.0,17.0,12.0,10.0,25.0,52.0,15.0,...,14.0,11.0,18.0,13.0,6.0,16.0,4.0,4.0,47.0,20.0
4,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,14.0,6.0,18.0,34.0,54.0,17.0,26.0,52.0,69.0,42.0,...,31.0,47.0,27.0,24.0,10.0,34.0,16.0,8.0,36.0,41.0
598,2.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,2.0,8.0,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [210]:
pred_matrix=Accesson_gene_matrix.dot(pred.T)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.020653,0.046116,0.032355,0.016299,0.045834,0.020118,0.036483,0.024179,0.048378,0.018032,...,0.047757,0.032819,0.024319,0.022314,0.072098,0.044635,0.023797,0.048049,0.050633,0.015656
1,1.717482,3.249690,1.749043,4.543196,2.772228,1.781432,2.033946,1.462870,2.573546,1.748234,...,4.219038,3.956580,1.732924,1.967405,4.480826,2.983322,2.636631,3.509736,2.223478,0.695955
2,0.021792,0.044821,0.055940,0.067368,0.070585,0.024764,0.058760,0.035387,0.092905,0.035424,...,0.059951,0.056072,0.048503,0.026488,0.072887,0.054049,0.049815,0.042986,0.107362,0.025002
3,1.428043,2.721459,1.425715,3.671139,2.208634,1.418406,1.615822,1.212739,2.107292,1.411179,...,3.478761,3.393594,1.443540,1.652774,3.655568,2.448257,2.165160,2.876818,1.787329,0.575542
4,0.036171,0.098035,0.027900,0.166886,0.075445,0.049158,0.056894,0.021967,0.075899,0.034627,...,0.135003,0.132635,0.037053,0.049175,0.158142,0.087511,0.074382,0.127527,0.047105,-0.005389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-0.006967,-0.012772,-0.006936,-0.003470,-0.011439,-0.007768,-0.008982,-0.005051,-0.009958,-0.005987,...,-0.014184,-0.006358,-0.006772,-0.006989,-0.015922,-0.012064,-0.007901,-0.012980,-0.008723,-0.002959
596,0.005604,-0.001278,-0.000161,-0.002192,-0.003058,0.001546,-0.002702,-0.000542,-0.001935,-0.000644,...,-0.001805,0.002160,-0.000681,0.005311,-0.006058,-0.001463,-0.000753,-0.001949,-0.001266,0.001007
597,1.686225,3.595369,2.028611,4.784174,3.027684,1.792111,2.298162,1.518334,3.097363,1.874728,...,4.787648,4.038159,2.013431,2.030965,5.036612,3.329056,2.968671,3.932011,2.753590,0.696128
598,0.141506,0.297416,0.227083,0.413882,0.283074,0.188199,0.229216,0.140720,0.294114,0.210936,...,0.410015,0.293949,0.209702,0.176863,0.431708,0.282927,0.267845,0.355095,0.276092,0.086600


In [211]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [212]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [213]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.13657990364244016

In [214]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.9968802821373196

In [215]:
pccs_row_df.to_csv("../results/Accesson/CMAE_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/CMAE_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [216]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
CMAE_cmd_accesson=CMD_dist(corr_pred,corr_true)

In [217]:
CMAE_cmd_accesson

0.1582534420465116

In [218]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
CMAE_cmd_cell=CMD_dist(corr_pred,corr_true)

In [219]:
CMAE_cmd_cell

0.000757286350439812

In [220]:
#DANCE
true=pd.read_csv("../results/ScMoGCN/ScMoGCN_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/ScMoGCN/ScMoGCN_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)
pred=pred.T
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [222]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix.dot(true)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,17.0,25.0,41.0,18.0,14.0,35.0,35.0,11.0,...,40.0,32.0,14.0,6.0,6.0,16.0,18.0,8.0,32.0,17.0
2,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,4.0,0.0,...,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
3,6.0,6.0,6.0,26.0,17.0,12.0,10.0,25.0,52.0,15.0,...,14.0,11.0,18.0,13.0,6.0,16.0,4.0,4.0,47.0,20.0
4,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,14.0,6.0,18.0,34.0,54.0,17.0,26.0,52.0,69.0,42.0,...,31.0,47.0,27.0,24.0,10.0,34.0,16.0,8.0,36.0,41.0
598,2.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,2.0,8.0,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [223]:
pred_matrix=Accesson_gene_matrix.dot(pred)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.088452,0.065836,0.078070,0.083087,0.069023,0.088652,0.073734,0.089568,0.080774,0.077536,...,0.073655,0.080034,0.088938,0.079960,0.060405,0.074113,0.085003,0.063661,0.087128,0.099697
1,7.054323,6.803593,6.851643,6.933008,6.806514,6.983245,6.892295,7.060219,6.832405,6.941525,...,6.878014,6.920863,6.942617,6.962204,6.788667,6.837365,6.993429,6.857829,6.968155,7.146766
2,0.230379,0.209883,0.201881,0.248181,0.222059,0.222923,0.201452,0.202833,0.225507,0.227783,...,0.228558,0.229036,0.214875,0.210836,0.198231,0.231136,0.211405,0.213149,0.213014,0.207378
3,5.700278,5.868319,5.795550,5.806452,5.848691,5.709932,5.797570,5.626273,5.768004,5.748521,...,5.843358,5.808144,5.669672,5.742708,5.837016,5.899563,5.713268,5.838553,5.696968,5.545598
4,-0.066044,-0.092691,-0.087661,-0.092831,-0.083261,-0.071540,-0.086116,-0.061874,-0.104408,-0.088225,...,-0.102097,-0.088805,-0.077918,-0.079978,-0.093296,-0.092237,-0.074897,-0.084809,-0.088914,-0.048402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-0.315248,-0.311914,-0.314361,-0.312398,-0.309599,-0.316036,-0.316241,-0.310849,-0.315072,-0.314298,...,-0.314712,-0.316828,-0.312650,-0.313221,-0.308385,-0.316029,-0.311762,-0.313100,-0.315096,-0.311282
596,-0.333361,-0.314441,-0.320241,-0.327081,-0.316574,-0.327280,-0.327064,-0.331257,-0.324350,-0.335166,...,-0.323267,-0.325413,-0.336404,-0.331387,-0.310075,-0.320387,-0.327207,-0.315157,-0.331896,-0.332635
597,4.073580,4.249563,4.095849,4.294621,4.158026,4.138635,4.086671,4.072260,4.288924,4.155316,...,4.319177,4.227565,4.138485,4.201837,4.345014,4.214227,4.196923,4.312105,4.200489,3.917101
598,-2.919896,-3.124705,-3.026150,-3.019879,-3.086904,-2.965570,-3.030487,-2.942118,-3.057261,-3.015492,...,-3.074162,-3.051686,-2.991261,-3.004200,-3.145003,-3.072550,-2.996200,-3.112183,-2.998160,-2.872156


In [224]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [225]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [226]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.003515083193564756

In [227]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

-0.5582733505433581

In [228]:
pccs_row_df.to_csv("../results/Accesson/ScMoGCN_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/ScMoGCN_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [229]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
DANCE_cmd_accesson=CMD_dist(corr_pred,corr_true)

In [230]:
DANCE_cmd_accesson

0.9952957620710172

In [231]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
DANCE_cmd_cell=CMD_dist(corr_pred,corr_true)

In [232]:
DANCE_cmd_cell

0.0002884985987966493

In [233]:
#KAUST
true=pd.read_csv("../results/LS_Lab/LS_Lab_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/LS_Lab/LS_Lab_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [234]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix.dot(true)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,17.0,25.0,41.0,18.0,14.0,35.0,35.0,11.0,...,40.0,32.0,14.0,6.0,6.0,16.0,18.0,8.0,32.0,17.0
2,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,4.0,0.0,...,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
3,6.0,6.0,6.0,26.0,17.0,12.0,10.0,25.0,52.0,15.0,...,14.0,11.0,18.0,13.0,6.0,16.0,4.0,4.0,47.0,20.0
4,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,14.0,6.0,18.0,34.0,54.0,17.0,26.0,52.0,69.0,42.0,...,31.0,47.0,27.0,24.0,10.0,34.0,16.0,8.0,36.0,41.0
598,2.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,2.0,8.0,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [235]:
pred_matrix=Accesson_gene_matrix.dot(pred)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.072181,0.032853,0.121606,0.049787,0.078955,0.085726,0.079020,0.056535,0.105183,0.045079,...,0.033734,0.044549,0.037209,0.074631,0.040265,0.066452,0.091962,0.062774,0.101813,0.112905
1,19.905365,11.140043,32.637464,17.885917,22.697462,23.891507,20.653227,17.790936,26.996437,15.841341,...,12.305159,13.867628,12.893742,21.140774,13.772928,19.350386,23.554723,17.245381,26.392331,28.554512
2,0.762580,0.387388,1.340938,0.600298,0.722167,0.887100,0.896524,0.511815,0.930823,0.551753,...,0.399501,0.481564,0.442419,0.639572,0.396629,0.617020,0.837972,0.579698,1.053346,1.054422
3,14.606125,8.612494,23.846818,14.614558,16.666281,17.534904,15.352683,14.011662,19.846510,12.397623,...,9.766640,10.555880,10.324483,15.947984,10.275394,14.283269,17.566492,12.937052,19.234946,21.068520
4,0.646825,0.344173,1.153721,0.588979,0.715862,0.774247,0.686895,0.556123,0.906432,0.521132,...,0.409510,0.455851,0.437202,0.654816,0.462953,0.624421,0.786139,0.550557,0.902674,0.990781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.034384,0.013643,0.057191,0.026086,0.042778,0.037453,0.028517,0.029669,0.050253,0.025868,...,0.019770,0.028480,0.027630,0.038847,0.026685,0.031047,0.052406,0.033458,0.049855,0.054700
596,0.022388,0.028783,0.042993,0.063151,-0.002726,0.023422,0.021873,0.030337,0.045017,0.037960,...,0.033495,0.035712,0.041375,0.052182,0.010815,-0.012492,0.025496,0.023619,0.045701,0.028086
597,25.652861,14.886518,41.637142,23.866795,29.196332,30.651979,27.142417,23.545934,34.972395,20.508842,...,16.619237,18.705710,17.172833,27.645389,18.011776,24.651728,30.710362,22.801450,34.104321,36.710328
598,1.528796,0.881238,2.193494,1.290882,1.691462,1.717309,1.512335,1.293447,2.056266,1.162920,...,0.937888,1.114902,0.988508,1.602576,1.030455,1.391861,1.798250,1.360083,1.944174,2.025209


In [236]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [237]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [238]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.2503725807659552

In [239]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.9978542379171084

In [240]:
pccs_row_df.to_csv("../results/Accesson/LS_Lab_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/LS_Lab_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [241]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
KAUST_cmd_accesson=CMD_dist(corr_pred,corr_true)

In [242]:
KAUST_cmd_accesson

0.11463004040116198

In [243]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
KAUST_cmd_cell=CMD_dist(corr_pred,corr_true)

In [244]:
KAUST_cmd_cell

0.000287075478156984

In [246]:
#MultiVI
true=pd.read_csv("../results/MultiVI/MultiVI_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/MultiVI/MultiVI_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)

indexes=[pred.index[i].split('_')[0] for i in range(len(pred.index))]
pred.index=indexes
peak_trim=true.columns.values.flatten().tolist()
cell_trim=true.index.values.flatten().tolist()

pred=pred.loc[cell_trim]
idx = np.unique(pred.index, return_index=True)[1]
pred=pred.iloc[idx]
overlap=list(set(pred.columns).intersection(set(true.columns)))
pred=pred.loc[:,overlap]
true=true.loc[:,overlap]
pred=pred.T.sort_index()
true=true.T.sort_index()

In [249]:
Accesson_gene_matrix_1 = Accesson_gene_matrix.loc[:,true.index]

In [250]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix_1.dot(true)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,17.0,25.0,41.0,18.0,14.0,35.0,35.0,11.0,...,40.0,32.0,14.0,6.0,6.0,16.0,18.0,8.0,32.0,17.0
2,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,4.0,0.0,...,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
3,6.0,6.0,6.0,26.0,17.0,12.0,10.0,25.0,52.0,15.0,...,14.0,11.0,18.0,13.0,6.0,16.0,4.0,4.0,47.0,20.0
4,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,14.0,6.0,18.0,34.0,54.0,17.0,26.0,52.0,69.0,42.0,...,31.0,47.0,27.0,24.0,10.0,34.0,16.0,8.0,36.0,41.0
598,2.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,2.0,8.0,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [251]:
pred_matrix=Accesson_gene_matrix_1.dot(pred)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.114343,0.080876,0.111578,0.058688,0.106766,0.079221,0.124251,0.118162,0.123015,0.060544,...,0.061956,0.057554,0.073219,0.078808,0.054389,0.147897,0.099937,0.085856,0.183823,0.120420
1,19.740252,17.766543,21.799829,20.587141,20.266545,18.184312,22.473564,20.120080,21.396697,17.439982,...,17.444426,14.765574,17.958278,17.292007,15.452509,22.969395,18.086403,18.005048,23.136587,20.411992
2,0.787805,0.725257,1.425156,0.902732,0.743361,1.006267,1.394675,0.693242,0.936181,0.813342,...,0.790624,0.510069,0.776320,0.493162,0.663782,0.972830,0.833207,0.777334,1.296310,1.114481
3,14.307701,13.364831,14.325038,15.978112,14.335438,12.652746,14.991967,16.687962,15.131055,13.015246,...,13.532262,12.089822,12.863537,13.870127,11.321458,15.956860,13.211763,13.093159,16.013475,14.272027
4,0.694330,0.637309,0.664202,0.780483,0.794718,0.578440,0.724040,0.789129,0.794400,0.510300,...,0.542936,0.464654,0.553758,0.682715,0.504167,0.814882,0.562522,0.610629,0.736516,0.643518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.047002,0.050756,0.069334,0.054254,0.075382,0.046090,0.067990,0.050173,0.060956,0.047895,...,0.043585,0.024584,0.074195,0.045018,0.033752,0.088596,0.038126,0.040853,0.071642,0.051132
596,0.036511,0.069125,0.030436,0.118853,0.023629,0.025338,0.033036,0.122825,0.033240,0.086448,...,0.124263,0.087017,0.047318,0.068281,0.025915,0.025757,0.048307,0.030053,0.051563,0.042527
597,28.862921,25.776676,29.890944,31.660255,29.218906,25.236844,30.842146,32.604123,30.558776,24.314569,...,25.019704,22.783583,24.562789,27.227700,21.909417,32.975964,26.926662,25.782316,33.829703,29.490281
598,1.910491,1.579152,1.937661,1.926192,1.945303,1.697299,2.021681,1.864404,1.949453,1.590235,...,1.558218,1.428225,1.681660,1.636109,1.459584,2.184082,1.785352,1.722907,2.184730,1.928183


In [252]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

/public/home/maintain/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]


In [253]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [254]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

nan

In [255]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.9977712103745007

In [256]:
pccs_row_df.to_csv("../results/Accesson/MultiVI_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/MultiVI_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [257]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
MultiVI_cmd_accesson=CMD_dist(corr_pred,corr_true)

/public/home/maintain/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [258]:
MultiVI_cmd_accesson

nan

In [259]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
MultiVI_cmd_cell=CMD_dist(corr_pred,corr_true)

In [260]:
MultiVI_cmd_cell

0.0002871156008433795

In [261]:
#BABEL
true=pd.read_csv("../results/BABEL/BABEL_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/BABEL/BABEL_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)
true=true.T
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [262]:
true

,chr1-100028620-100029223,chr1-100034488-100034828,chr1-100036915-100039547,chr1-100046586-100047303,chr1-100132285-100133776,chr1-100184590-100184896,chr1-100249022-100250476,chr1-100265455-100266982,chr1-100299590-100299838,chr1-100307011-100307158,...,chrY-11314266-11315078,chrY-11322458-11322849,chrY-11327423-11327918,chrY-11329701-11330187,chrY-11332886-11334319,chrY-11721792-11722236,chrY-11747573-11747839,chrY-56829443-56829881,chrY-56830756-56834738,chrY-56835728-56837353
AAACAGCCAAAGCGGC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAACAGGAT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAAGGTACG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAGGATGGC-1,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCATTCAGCA-1,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGACCTGG-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTGCATTAG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTGCTCCAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTGGACCTG-1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [263]:
#得到gene cell矩阵 
true_matrix=Accesson_gene_matrix.dot(true.T)
true_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,17.0,25.0,41.0,18.0,14.0,35.0,35.0,11.0,...,40.0,32.0,14.0,6.0,6.0,16.0,18.0,8.0,32.0,17.0
2,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,4.0,0.0,...,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
3,6.0,6.0,6.0,26.0,17.0,12.0,10.0,25.0,52.0,15.0,...,14.0,11.0,18.0,13.0,6.0,16.0,4.0,4.0,47.0,20.0
4,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,14.0,6.0,18.0,34.0,54.0,17.0,26.0,52.0,69.0,42.0,...,31.0,47.0,27.0,24.0,10.0,34.0,16.0,8.0,36.0,41.0
598,2.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,2.0,8.0,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [264]:
pred_matrix=Accesson_gene_matrix.dot(pred.T)
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,14.619056,8.628160,16.260971,15.851872,13.551689,11.371943,16.013744,14.649243,20.045268,11.952515,...,10.236055,11.919480,13.965235,11.102449,11.897040,14.189364,10.984307,11.526911,18.015047,18.205699
2,0.331147,0.120059,0.558791,0.338546,0.367449,0.344375,0.602527,0.327596,0.744757,0.342656,...,0.179758,0.083094,0.350610,0.211907,0.235835,0.499801,0.348647,0.341049,0.618030,0.492821
3,12.020661,7.754430,13.076581,14.344847,11.618522,9.228274,12.984397,12.629363,16.305082,10.065969,...,9.077261,11.172061,11.820224,9.790242,10.872098,11.944935,9.004490,9.921948,14.572034,14.733373
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
597,17.434457,10.640322,18.840178,22.800718,17.020703,12.672750,18.962069,17.985966,24.413115,13.720415,...,12.834918,16.281290,16.325353,13.641430,15.823414,17.425458,12.820174,14.477355,21.172051,21.579837
598,1.138413,0.711880,1.218860,1.665108,1.081262,0.848562,1.228015,1.173976,1.566206,0.934672,...,0.893350,1.104207,1.077821,0.925546,0.977584,1.126735,0.863199,0.933671,1.348976,1.378889


In [265]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

/public/home/maintain/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/public/home/maintain/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [266]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [267]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.nanmedian(pccs_row_df)

0.363116683184514

In [268]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.9974309635934592

In [269]:
pccs_row_df.to_csv("../results/Accesson/BABEL_" + train_id + "_" + test_id + "_accesson_pcc.csv")
pccs_col_df.to_csv("../results/Accesson/BABEL_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [270]:
pred_matrix

,AAACAGCCAAAGCGGC-1,AAACAGCCAACAGGAT-1,AAACAGCCAAGGTACG-1,AAACAGCCAGGATGGC-1,AAACAGCCATTCAGCA-1,AAACATGCACAAACTT-1,AAACCAACAAGGGTTG-1,AAACCAACAGCAAGTG-1,AAACCAACATAAAGCA-1,AAACCGAAGCACCACA-1,...,TTTGTGTTCCGCAAAT-1,TTTGTGTTCGTTCACC-1,TTTGTGTTCTAATCAG-1,TTTGTGTTCTCAATAG-1,TTTGTTGGTAAATTGC-1,TTTGTTGGTGACCTGG-1,TTTGTTGGTGCATTAG-1,TTTGTTGGTGCTCCAC-1,TTTGTTGGTGGACCTG-1,TTTGTTGGTTAAGCCA-1
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,14.619056,8.628160,16.260971,15.851872,13.551689,11.371943,16.013744,14.649243,20.045268,11.952515,...,10.236055,11.919480,13.965235,11.102449,11.897040,14.189364,10.984307,11.526911,18.015047,18.205699
2,0.331147,0.120059,0.558791,0.338546,0.367449,0.344375,0.602527,0.327596,0.744757,0.342656,...,0.179758,0.083094,0.350610,0.211907,0.235835,0.499801,0.348647,0.341049,0.618030,0.492821
3,12.020661,7.754430,13.076581,14.344847,11.618522,9.228274,12.984397,12.629363,16.305082,10.065969,...,9.077261,11.172061,11.820224,9.790242,10.872098,11.944935,9.004490,9.921948,14.572034,14.733373
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
597,17.434457,10.640322,18.840178,22.800718,17.020703,12.672750,18.962069,17.985966,24.413115,13.720415,...,12.834918,16.281290,16.325353,13.641430,15.823414,17.425458,12.820174,14.477355,21.172051,21.579837
598,1.138413,0.711880,1.218860,1.665108,1.081262,0.848562,1.228015,1.173976,1.566206,0.934672,...,0.893350,1.104207,1.077821,0.925546,0.977584,1.126735,0.863199,0.933671,1.348976,1.378889


In [271]:
pred_matrix=pred_matrix.loc[(pred_matrix!=0).any(axis=1)]

In [272]:
true_matrix=true_matrix.loc[pred_matrix.index]

In [273]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
BABEL_cmd_accesson=CMD_dist(corr_pred,corr_true)

In [274]:
BABEL_cmd_accesson

0.07676349929257842

In [275]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
BABEL_cmd_cell=CMD_dist(corr_pred,corr_true)

In [276]:
BABEL_cmd_cell

0.0002909511867386083

In [277]:
data_cell = [['LIGER',LIGER_cmd_cell],['Seurat',Seurat_cmd_cell],['CMAE',CMAE_cmd_cell],['BABEL',BABEL_cmd_cell],['DANCE',DANCE_cmd_cell],['KAUST',KAUST_cmd_cell],['MultiVI',MultiVI_cmd_cell]]
df_cell = pd.DataFrame(data_cell,columns=['methods','cmd'])

In [278]:
data_accesson = [['LIGER',LIGER_cmd_accesson],['Seurat',Seurat_cmd_accesson],['CMAE',CMAE_cmd_accesson],['BABEL',BABEL_cmd_accesson],['DANCE',DANCE_cmd_accesson],['KAUST',KAUST_cmd_accesson],['MultiVI',MultiVI_cmd_accesson]]
df_accesson = pd.DataFrame(data_accesson,columns=['methods','cmd'])

In [279]:
df_accesson.to_csv("../results/Accesson/" + train_id + "_" + test_id + "_accesson_cmd.csv")
df_cell.to_csv("../results/Accesson/" + train_id + "_" + test_id + "_cell_cmd.csv")